### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import yaml
import os
from ads.opctl.operator.lowcode.forecast.__main__ import operate, ForecastOperatorConfig
from ads import set_auth

set_auth("resource_principal")

style = {'description_width': 'initial', 'button_color': 'lightblue'}
layout = widgets.Layout(width='90%')

# Title
title = widgets.HTML(
    value="<h1>AI Forecasting Operator</h1>",
)

# Initialize widgets for the form fields
historical_data_uri = widgets.Text(
    placeholder="Enter URI for historical data",
    description="Historical Data URI:",
    style=style,
    layout=layout,
)
additional_data_uri = widgets.Text(
    placeholder="Enter URI for additional data",
    description="Additional Data URI:",
    style=style,
    layout=layout,
)
model_dropdown = widgets.Dropdown(
    options=["Prophet", "NeuralProphet", "AutoMLX", "Auto-Select", "AutoTS", "Arima"],
    value="Prophet",
    description="Model:",
    style=style,
    # layout=layout,
)
horizon_input = widgets.IntText(
    value=1,
    description="Horizon:",
    style=style,
    # layout=layout,
)
target_column_input = widgets.Text(
    placeholder="Enter target column name",
    description="Target Column:",
    style=style,
    # layout=layout,
)
datetime_name_input = widgets.Text(
    placeholder="Enter datetime column name",
    description="Datetime Column:",
    style=style,
    # layout=layout,
)
datetime_format_input = widgets.Text(
    placeholder="Enter datetime format (e.g., %Y-%m-%d)",
    description="Datetime Format:",
    style=style,
    # layout=layout,
)
output_yaml_filename = widgets.Text(
    placeholder="e.g. forecast.yaml",
    description="Filename for forecast configuration yaml",
    style=style,
    layout=layout,
)

generate_explanations = widgets.Checkbox(value=False, description='Generate Explanations', style=style)
output_dir = widgets.Text(value="results", description='Output Directory', style=style)
target_category_column = widgets.Text(description='Target Category Column', style=style)
confidence_interval_width = widgets.BoundedFloatText(value=0, min=0, max=10, description='Number of Tuning Trials:', style=style)
tuning = widgets.BoundedIntText(value=0, min=0, max=10, description='Number of Tuning Trials:', style=style)
metric = widgets.Dropdown(
    options=["RMSE", "MAPE", "SMAPE"],
    value="SMAPE",
    description="Metric:",
    style=style,
)

# Group them inside a VBox
additional_options_content = widgets.VBox([output_dir, target_category_column, generate_explanations, metric, tuning], style=style)

# Create an Accordion
additional_options = widgets.Accordion(children=[additional_options_content], style=style)
additional_options.set_title(0, "Additional Options")

# Widgets for rendering data
historical_data_output = widgets.Output()
datetime_preview_output = widgets.Output()

# Load historical data and render table
def load_historical_data(_):
    with historical_data_output:
        historical_data_output.clear_output()
        try:
            # Simulating data loading from a URI
            # Replace the below line with your actual data loading logic
            uri = historical_data_uri.value
            data = pd.read_csv(uri)  # Assume URI is a CSV file path for simplicity
            display(data.head())  # Show the first 10 rows
        except Exception as e:
            print(f"Error loading data from URI: {e}")

# Show preview of datetime column with specified format
def preview_datetime_column(_):
    with datetime_preview_output:
        datetime_preview_output.clear_output()
        try:
            # Simulate data processing
            # Replace the below line with logic to fetch and parse your datetime column
            uri = historical_data_uri.value
            data = pd.read_csv(uri)
            datetime_col = datetime_name_input.value
            datetime_format = datetime_format_input.value or None
            data[datetime_col] = pd.to_datetime(data[datetime_col], format=datetime_format)
            data["Interpretted Datetime"] = data[datetime_col].dt.strftime('%c')
            display(data[[datetime_col, "Interpretted Datetime"]].head())
        except Exception as e:
            print(f"Error processing datetime column: {e}")

# Buttons for actions
load_data_button = widgets.Button(description="Load Historical Data", style=style)
load_data_button.on_click(load_historical_data)

preview_datetime_button = widgets.Button(description="Preview Datetime Column", style=style)
preview_datetime_button.on_click(preview_datetime_column)

# Assemble the form layout
form = widgets.VBox([
    title,
    historical_data_uri,
    load_data_button,
    historical_data_output,
    additional_data_uri,
    model_dropdown,
    horizon_input,
    target_column_input,
    datetime_name_input,
    datetime_format_input,
    preview_datetime_button,
    datetime_preview_output,
    output_yaml_filename,
    additional_options,
])

# Display the form
display(form)

# Function to export form inputs as YAML
def export_to_dict():
    spec = {
        "kind": "operator",
        "type": "forecast",
        "spec": {
            "historical_data": {"url": historical_data_uri.value},
            "model": model_dropdown.value,
            "horizon": horizon_input.value,
            "target_column": target_column_input.value,
            "datetime_column": {
                "name": datetime_name_input.value,
            }
        }
    }
    if additional_data_uri.value:
        spec["spec"]["additional_data"] = {"url": additional_data_uri.value}
    if datetime_format_input.value:
        spec["spec"]["datetime_column"]["format"] = datetime_format_input.value
    if target_category_column.value:
        spec["spec"]["target_category_columns"] = [target_category_column.value]
    return spec

def export_to_yaml():
    return yaml.dump(export_to_dict(), sort_keys=False)

def get_forecast_yaml_filename():
    if output_yaml_filename.value:
        return output_yaml_filename.value
    filename = "forecast.yaml"
    i = 0
    while os.path.exists(filename):
        i = i + 1
        filename = f"forecast_{i}.yaml"
    return filename

# Example: Export YAML
export_button = widgets.Button(description="Save", style=style)
def export_yaml_action(_):
    yaml_exp = ""
    with open(get_forecast_yaml_filename(), 'w') as f:
        yaml_exp = export_to_yaml()
        f.write(yaml_exp)
export_button.on_click(export_yaml_action)

# Run
run_local_button = widgets.Button(description="Run Local", style=style)
def run_local_action(_):
    config = ForecastOperatorConfig.from_yaml(export_to_yaml())
    operate(config)
run_local_button.on_click(run_local_action)

from ads.opctl.cmds import run
run_job_button = widgets.Button(description="Run Job", style=style)
def run_job_action(_):
    config_dict = export_to_dict()
    config_dict["execution"] = {"backend": "job"}
    print(config_dict)
    run(config_dict)
run_job_button.on_click(run_job_action)

# Add export button to the form
form.children += (widgets.HBox((export_button, run_local_button, run_job_button)),)


In [2]:
%%writefile forecast-from_jupyter.yaml
kind: operator
type: forecast
version: v1
spec:
    datetime_column:
        name: ds
    historical_data:
        url: https://raw.githubusercontent.com/facebook/prophet/main/examples/example_yosemite_temps.csv
    horizon: 3
    model: prophet
    target_column: y

Overwriting forecast-from_jupyter.yaml


In [3]:
!ads operator run -f forecast-from_jupyter.yaml --debug

DEBUG:ads.opctl:/home/datascience/.ads_ops/config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/ml_job_config.ini does not exist. No config loaded.
INFO:ads.opctl:Backend config is not provided, the local will be used by default. 
DEBUG:ads.opctl:/home/datascience/.ads_ops/config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/ml_job_config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/ml_job_config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/config.ini does not exist. No config loaded.
DEBUG:ads.opctl:/home/datascience/.ads_ops/ml_job_config.ini does not exist. No config loaded.
INFO:ads.opctl:************************************************** Runtime Config **************************************************
INFO:ads.opctl:
kind: operator.local
spec: null
type: